In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
Base = declarative_base()
engine = create_engine('sqlite:///Mood2Food.db')

In [3]:
class ChooseMood(Base):
    __tablename__ = 'choosemood'
    mood = Column(String(15), primary_key = True)
    color = Column(String(20), nullable = False)
    color_hex = Column(String(7), nullable = False)
    color_r = Column(Integer, nullable = False)
    color_g = Column(Integer, nullable = False)
    color_b = Column(Integer, nullable = False)

class Mood2Category(Base):
    __tablename__ = 'mood2category'
    category = Column(String(10), primary_key = True)
    mood = Column(String(15), ForeignKey("choosemood.mood"))
    act_mood = relationship(ChooseMood)
    
class Category2Food(Base):
    __tablename__ = 'category2food'
    food = Column(String(50), primary_key = True)
    category = Column(String(10), ForeignKey("mood2category.category"))
    ingredients = Column(String(1000))
    directions = Column(String(1000))
    url = Column(String(100))
    act_category = relationship(Mood2Category)

Base.metadata.create_all(engine)

In [4]:
moods = pd.read_csv("Data/Moods.csv")
moods.head()

,emotion,colors,Hex Code#RRGGBB,"Decimal Code (R,G,B)"
0,happy,yellow,#FFFF00,"(255,255,0)"
1,sad,blue,#6495ED,"-100,149,237"
2,calm,green,#90EE90,"-144,238,144"
3,excited,red,#FF4500,"(255,69,0)"
4,anxious,red,#FF0000,"(255,0,0)"


In [5]:
moods[["R", "G", "B"]] = moods["Decimal Code\r\n(R,G,B)"].str.replace("[\)\(]", "").str.split(",", expand = True)
moods = moods.drop(columns=["Decimal Code\r\n(R,G,B)"])

In [6]:
moods.columns = ["mood", "color", "color_hex", "color_r", "color_g", "color_b"]

In [7]:
moods.head()

,mood,color,color_hex,color_r,color_g,color_b
0,happy,yellow,#FFFF00,255,255,0
1,sad,blue,#6495ED,-100,149,237
2,calm,green,#90EE90,-144,238,144
3,excited,red,#FF4500,255,69,0
4,anxious,red,#FF0000,255,0,0


In [15]:
category = pd.read_csv("Data/Mood2Category.csv")
category = category[["Category", "Mood"]]
category["Mood"] = category["Mood"].str.lower()
category.head()

,Category,Mood
0,Meat,angry
1,Tough,angry
2,Crunchy,angry
3,Sugar,sad
4,Sweet,anxious


In [16]:
food = pd.read_csv("Data/Category2Food.csv")
food.columns = ["category", "food", "ingredients", "directions", "url"]
food = food[["food", "category", "ingredients", "directions", "url"]]
food.head()

,food,category,ingredients,directions,url
0,Beef Brisket,Meat,3 to 4 lbs of a brisket cut of beef\r\n3/4 cup...,1. Trim the fat cap on the brisket down to abo...,https://www.simplyrecipes.com/recipes/beef_bri...
1,Beef & Noodles,Meat,NaN,NaN,https://www.yummly.com/recipe/Lazy-Beef-_-Nood...
2,Firecracker Chicken,Meat,NaN,NaN,https://www.yummly.com/recipe/Firecracker-Chic...
3,Hawaiian Meatballs,Meat,NaN,NaN,https://www.yummly.com/recipe/Hawaiian-Meatbal...
4,Hidden Valley Candied Nuts,Tough,NaN,NaN,https://www.yummly.com/recipe/Hidden-Valley-Ca...


In [17]:
Session = sessionmaker(bind=engine)
session = Session()

In [18]:
db_moods = {}
for i in range(len(moods)):
    db_moods[moods["mood"][i]] = ChooseMood(mood = moods["mood"][i],
              color = moods["color"][i],
              color_hex = moods["color_hex"][i],
              color_r = moods["color_r"][i],
              color_g = moods["color_g"][i],
              color_b = moods["color_b"][i])

In [19]:
db_categories = {}
for i in range(len(category)):
    db_categories[category["Category"][i]] = Mood2Category(category = category["Category"][i],
                                                           mood = category["Mood"][i],
                                                           act_mood = db_moods[category["Mood"][i]])

In [20]:
db_foods = {}
for i in range(len(food)):
    db_foods[food["food"][i]] = Category2Food(food = food["food"][i],
                                              category = food["category"][i],
                                              ingredients = food["ingredients"][i],
                                              directions = food["directions"][i],
                                              url = food["url"][i],
                                              act_category = db_categories[food["category"][i]])

In [21]:
db_categories.keys()

dict_keys(['Meat', 'Tough', 'Crunchy', 'Sugar', 'Sweet', 'Soft', 'Salty', 'Fatty', 'Bulky', 'Starch', 'Combination', 'Asparagus', 'Avocado', 'Blueberries', 'Healthy', 'Chocolate', 'Protein', 'Vitamins', 'Chamomile', 'Oatmeal'])

In [22]:
session.add_all(db_moods.values())

In [23]:
session.commit()

In [24]:
session.add_all(db_categories.values())

In [25]:
session.commit()

In [26]:
session.add_all(db_foods.values())

In [27]:
session.commit()

In [28]:
session.close()